In [1]:
import mediapipe as mp
import numpy as np
import cv2
import math
import pandas as pd

In [2]:
facemesh =  mp.solutions.face_mesh
face = facemesh.FaceMesh(
    static_image_mode=False, 
    min_tracking_confidence=0.5, 
    min_detection_confidence=0.5
    )
draw = mp.solutions.drawing_utils


In [31]:
def EARandMAR(right_eye, left_eye, mouth):
    right_eye = (eucledianDistance(right_eye[2], right_eye[5]) + eucledianDistance(right_eye[4], right_eye[3]))/(2*eucledianDistance(right_eye[0], right_eye[1]))
    left_eye = (eucledianDistance(left_eye[4], left_eye[3]) + eucledianDistance(left_eye[5], left_eye[2]))/(2*eucledianDistance(left_eye[1], left_eye[0]))
    mouth = (eucledianDistance(mouth[0], mouth[2]) + eucledianDistance(mouth[3], mouth[5]))/(2*eucledianDistance(mouth[1], mouth[4]))
    return right_eye, left_eye, mouth

def eucledianDistance(point1, point2):
    return math.sqrt((point2.x - point1.x)**2 + (point2.y - point1.y)**2)

In [54]:
cap =  cv2.VideoCapture(1)
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [
    #upper right mouth
    # 82, 
    38, 
    # 72, 37,

    # 81, 41, 73, 39,
    # 80, 42, 74, 40,
    # 183,184, 185, 191,
    
    # 61, 
    76, 
    # 62, 78,

    #upper middle mouth
    # 14,  13, 0, 11,

    #upper left mouth
    # 312, 
    268, 
    # 302, 267,
    
    # 311, 271, 303, 269,
    # 310, 272, 304, 270,
    # 415, 407, 408, 409,
    
    # 291, 
    292,
    #  308, 306,
    
    # 324,325, 307, 375,

    #botom right mouth
    # 146, 77,
    # 88, 89,90, 91,
    # 178, 179, 180, 181,
    
    # 87, 
    86, 
    # 85, 84,

    #botom middle mouth
    # 12, 15, 16, 17,

    #botom left mouth
    # 318, 319, 320, 321,
    # 402, 403, 404, 405,
    
    # 317,
    316, 
    # 315, 314
    ]

face_data = {
    "right_eyes" : [],
    "left_eyes" : [],
    'mouth' : []
    }

while True:
    _, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgH, imgW, _ = frame.shape
    # print(imgH, imgW)
    rgb =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = face.process(rgb)
    if op.multi_face_landmarks:
        for pt in op.multi_face_landmarks:
            # draw.draw_landmarks(frame, i)
            temp = [[],[],[]]
            left = 1
            right = 1
            mouth = 1
            for no,point in enumerate(pt.landmark):
                if no in chosen_left_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, color, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{left}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[0].append(point)
                    left += 1
                if no in chosen_right_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (color, 0, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{right}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[1].append(point)
                    right += 1
                if no in chosen_mouth_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, 0, color), -1)
                    cv2.putText(
                        frame, text= str(f'p{mouth}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[2].append(point)
                    mouth += 1
            right_eye, left_eye, mouth = EARandMAR(temp[0], temp[1], temp[2])
            face_data['right_eyes'].append(right_eye)
            face_data['left_eyes'].append(left_eye)
            face_data['mouth'].append(mouth)
            if right_eye < 0.3 and left_eye < 0.3:
                cv2.putText(
                        frame, text= str(f'Driver Berkedip'), org=(20, 20),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,0,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
            else:
                cv2.putText(
                        frame, text= str(f'Driver Tidak Berkedip'), org=(20, 20),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
                
            if mouth > 0.5:
                cv2.putText(
                        frame, text= str(f'Driver Menguap'), org=(20, 40),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,0,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
            else:
                cv2.putText(
                        frame, text= str(f'Driver Tidak Menguap'), org=(20, 40),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("windows", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    # break

# df = pd.DataFrame(face_data)
print(face_data)
cap.release()
cv2.destroyAllWindows()

{'right_eyes': [0.0672599123857329, 0.029754736876369762, 0.2610302315727485, 0.34752745916140826, 0.3243678712536666, 0.3150666678971994, 0.31409449541323936, 0.3146564251398434, 0.31761635973715646, 0.3118337432253732, 0.3149451146080941, 0.3156455241557098, 0.31732668480577636, 0.3158044849281762, 0.32377203402237814, 0.3246763743406626, 0.32463422247972623, 0.3376202415308341, 0.3281262408511419, 0.3285183214908592, 0.3282254074923495, 0.33022481041017077, 0.3044561628151624, 0.3217796582491478, 0.3206843163935749, 0.32109898729012265, 0.34078865469222397, 0.33096531888052744, 0.33763813781790863, 0.32796636426508874, 0.3304913031269193, 0.3337751502497383, 0.3315781575093329, 0.32841693216792833, 0.32793412458816185, 0.32997533229576526, 0.32672162627783463, 0.33178977445551033, 0.3375545631912672, 0.3311081034603063, 0.3364419322205779, 0.355304040523705, 0.3407751682618559, 0.3436660144697797, 0.34634858182965544, 0.3175686521681834, 0.32382524892925685, 0.32262638147468836, 0.3

In [47]:
print(len(face_data['right_eyes']),len(face_data['left_eyes']),len(face_data['mouth']))
pd.DataFrame(face_data).head(60)

66 66 66


,right_eyes,left_eyes,mouth
0,0.367478,0.363447,0.120242
1,0.371013,0.367503,0.120842
2,0.395132,0.392043,0.096786
3,0.395918,0.393412,0.100082
4,0.400957,0.397047,0.097624
5,0.391583,0.385838,0.106755
6,0.390621,0.385544,0.104772
7,0.391369,0.386782,0.103907
8,0.391959,0.387406,0.103753
9,0.377262,0.370276,0.107150


In [14]:
# elif no > 200 and no < 500 :
                #     cv2.putText(
                #         frame, text= str(no), org=(int(point.x * imgW), int(point.y * imgH)),
                #         fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(0,255,0),
                #         thickness=1, lineType=cv2.LINE_AA
                #     )

import time

In [17]:
print(time.localtime())

time.struct_time(tm_year=2023, tm_mon=8, tm_mday=15, tm_hour=18, tm_min=3, tm_sec=3, tm_wday=1, tm_yday=227, tm_isdst=0)
